# Layered Flow Fields

This is based on a blog post from [Tyler Hobbs](https://tylerxhobbs.com/essays/2020/flow-fields) :)

In [ ]:
import os
import colorsys
import cv2
import numpy as np
import scipy.interpolate as sci
from cv2 import VideoWriter, VideoWriter_fourcc

width = 640
height = 480
fps = 15
seconds = 15
frames = int(fps * seconds)

# Could use "test.mp4" and the "H264" fourcc codec below.
FILENAME = "flow_fields.webm"
if os.path.exists(FILENAME):
    os.remove(FILENAME)

In [ ]:
blob_w, blob_h = width // 4, height // 4
blob_count = 20
TOP_SPEED = 20.5

# We'll have some blocks of various sizes make up blobs.
class Blob:
    def __init__(self, x: float, y: float, size: float):
        self.x = x
        self.y = y
        self.size = size
        # pick a random direction
        self.x_delta = (np.random.random() - 0.5) * TOP_SPEED / fps
        self.y_delta = (np.random.random() - 0.5) * TOP_SPEED / fps

    def move(self):
        self.x += self.x_delta
        self.y += self.y_delta

        if self.x < -self.size:
            self.x = blob_w + self.size
        elif self.x > blob_w + self.size:
            self.x = -self.size
            
        if self.y < -self.size:
            self.y = blob_h + self.size
        elif self.y > blob_h + self.size:
            self.y = -self.size

    def draw(self, img):
        cv2.circle(
            img,
            (int(round(self.x)), int(round(self.y))),
            self.size,
            (255, 255, 255),
            thickness=-1,
            lineType=cv2.LINE_AA,
        )


blobs = [
    Blob(
        np.random.randint(blob_w),
        np.random.randint(blob_h),
        np.random.randint(blob_w / 3),
    )
    for _ in range(blob_count)
]


In [ ]:
fourcc = VideoWriter_fourcc(*"VP80")
video = VideoWriter(FILENAME, fourcc, float(fps), (width, height))

resolution = width * 0.01
num_columns = int(width / resolution)
num_rows = int(height / resolution)

grid = np.ones((num_columns, num_rows))
lengths = resolution * 1.0 * np.ones((num_columns, num_rows))

default_angle = np.pi * 0.25

x_clr = 255.0 / num_columns
y_clr = 255.0 / num_rows

# Create grids for x and y
ys, xs = np.meshgrid(np.arange(num_rows), np.arange(num_columns))

# Compute pixel locations for all xs and ys
xs1 = xs * resolution
ys1 = ys * resolution

xs1_int = xs1.astype(int)
ys1_int = ys1.astype(int)

saturate = lambda arr, max_value: np.round(np.clip(arr, 0, max_value)).astype(int)
saturate_x = lambda arr: saturate(arr, width - 1)
saturate_y = lambda arr: saturate(arr, height - 1)

thickness = np.ones((num_columns, num_rows), dtype=np.uint8)

np.random.seed(123456)

mod1 = 0.0
mod2 = np.random.random()

# Rows and cols for flow lines.
v_rows = num_rows
v_cols = num_columns

# Current x and y:
pcursor_initial = np.zeros((v_cols, v_rows, 2))
for r in range(v_rows):
    for c in range(v_cols):
        # Initial x, y
        pcursor_initial[c][r][0] = np.random.uniform() * v_cols * resolution
        pcursor_initial[c][r][1] = np.random.uniform() * v_cols * resolution

for frame_num in range(frames):
    # Start with a white background.
    frame = np.ones((height, width, 3), dtype=np.uint8) * 255
    cos_grid = np.cos(grid)
    sin_grid = np.sin(grid)

    line_offsets_x = np.multiply(cos_grid, lengths)
    line_offsets_y = np.multiply(sin_grid, lengths)
    xs2 = np.round(xs1 + line_offsets_x).astype(int)
    ys2 = np.round(ys1 + line_offsets_y).astype(int)

    # Draw a curve
    num_steps = 45  # 150
    points = np.zeros((num_steps, 2))
    step_length = 0.25
    pcursor = np.array(pcursor_initial)

    # For each v_row, v_col, we want a list of points
    parr = np.zeros((v_cols, v_rows, num_steps, 2))

    for i in range(num_steps):
        # Assign cursor x, y to the step index.
        parr[:, :, i] = pcursor[:, :]

        x_index = saturate(pcursor[:, :, 0] / resolution, num_columns - 1)
        y_index = saturate(pcursor[:, :, 1] / resolution, num_rows - 1)

        # Now for the array of column and row indices, we want to
        # look up the sin and cos values
        x_step = step_length * cos_grid[x_index, y_index]
        y_step = step_length * sin_grid[x_index, y_index]

        # Increment the x, y offsets
        pcursor[:, :, 0] += x_step
        pcursor[:, :, 1] += y_step

    for r in range(v_rows):
        for c in range(v_cols):
            first_x, first_y = parr[c, r, 0, 0], parr[c, r, 0, 1]
            hsv_h = np.sin(mod1 + (first_x / v_cols))
            hsv_s = 0.6
            hsv_v = np.cos(mod2)
            rgb_r, rgb_g, rgb_b = colorsys.hsv_to_rgb(hsv_h, hsv_s, hsv_v)
            rgb = (
                int(np.round(rgb_r * 255)),
                int(np.round(rgb_b * 255)),
                int(np.round(rgb_b * 255)),
            )
            frame = cv2.polylines(
                frame,
                [np.round(parr[c, r, :, :]).astype(np.int32)],
                False,
                rgb,
                lineType=cv2.LINE_AA,
            )

    # Redraw blobs each time.
    blob_img = np.zeros((blob_h, blob_w, 3))
    for blob in blobs:
        blob.draw(blob_img)

    x = np.linspace(0, 1, blob_w, endpoint=False)
    y = np.linspace(0, 1, blob_h, endpoint=False)

    to_int = blob_img[:, :, 0]
    kernel = int(blob_w * 0.75)
    if kernel % 1 == 0:
        kernel += 1
    to_int = cv2.GaussianBlur(to_int, (kernel, kernel), 0.0)
    f = sci.interp2d(x, y, to_int, kind="cubic")

    znew = f(np.linspace(0, 1, num_rows), np.linspace(0, 1, num_columns))
    # normalize
    znew -= np.min(znew)
    znew /= np.max(znew)
    # we want znew to be 1.1 - 1.9 * pi
    grid[:, :] = ((znew * 0.8) + 1.1) * np.pi

    # Move blocks around
    for blob in blobs:
        blob.move()

    video.write(frame)

    # Mod ticks
    mod1 += 0.25 / fps
    mod2 += 0.3 / fps


video.release()

In [ ]:
from IPython.display import Video

Video(FILENAME, embed=True)